In [4]:
from transformers import pipeline
import google.generativeai as genai
from dotenv import load_dotenv
import os
import re
from fractions import Fraction
import json

# 1. Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# 2. Initialize models (CPU compatible)
# (Keep the NER pipeline if needed for other purposes)
food_ner = pipeline(
    "token-classification",
    model="Dizex/InstaFoodRoBERTa-NER",
    aggregation_strategy="simple",
    device=-1  # Force CPU usage explicitly
)
genai.configure(api_key=GEMINI_API_KEY)
gemini = genai.GenerativeModel("gemini-2.0-flash")

# 3. Enhanced quantity parser
def parse_quantity(qty_str):
    try:
        if ' ' in qty_str and '/' in qty_str:  # Handle "1 1/2"
            whole, fraction = qty_str.split()
            return float(whole) + float(Fraction(fraction))
        return float(Fraction(qty_str))
    except Exception as e:
        print(f"Quantity parse error: {e}")
        return None

# 4. Ingredient extractor using line-by-line regex parsing
def extract_ingredients(text):
    ingredients = []
    for line in text.splitlines():
        line = line.strip()
        # Look for lines that start with the marker '-'
        if line.startswith("-"):
            # Remove the starting '-' and trim spaces
            line_content = line.lstrip("-").strip()
            # Regex to capture quantity, unit, and ingredient description
            match = re.match(
                r'(\d+\s\d+/\d+|\d+/\d+|\d+\.\d+|\d+)\s*(cup|tbsp|tsp|oz|lb|teaspoon|tablespoon)s?\s+(.*)',
                line_content,
                flags=re.IGNORECASE
            )
            if match:
                qty, unit, ingredient = match.group(1), match.group(2), match.group(3)
                quantity = parse_quantity(qty)
                if quantity is not None:
                    ingredients.append({
                        "ingredient": ingredient.lower(),
                        "quantity": quantity,
                        "unit": unit.lower()
                    })
    return ingredients

# 5. Adjusted Gemini conversion
# 5. Adjusted Gemini conversion
def convert_with_gemini(ingredients):
    system_prompt = f"""
    Convert these baking ingredients to exact grams or milliletres. Return ONLY JSON:

    Rules:
    - Use professional baking standards.
    - For dry ingredients (flour, sugar), specify if packed/loose.
    - Return format:
    {{
      "ingredient": string,
      "grams": number, (if dry)
      "ml": number, (if wet)
    }}

    Input to convert:
    {json.dumps(ingredients, indent=2)}
    """
    
    try:
        response = gemini.generate_content(system_prompt + "\nInput:\n" + json.dumps(ingredients))
        response_text = response.text.strip()
        
        # If the response is wrapped in triple backticks with json formatting, remove them.
        if response_text.startswith("```json"):
            response_text = re.sub(r"^```json|```$", "", response_text, flags=re.DOTALL).strip()
        
        # As a fallback, ensure the JSON begins with '{' or '[' by extracting from the first brace
        if not response_text.startswith("{") and not response_text.startswith("["):
            first_brace = response_text.find("{")
            if first_brace == -1:
                first_brace = response_text.find("[")
            if first_brace != -1:
                response_text = response_text[first_brace:]
        
        if not response_text:
            raise ValueError("Empty response from Gemini API.")
        
        return json.loads(response_text)
    except Exception as e:
        print(f"Gemini Error: {str(e)}")
        return None


# 6. Main processor with debug info
def process_recipe(text):
    print("🔍 Extracting ingredients from recipe lines...")
    ingredients = extract_ingredients(text)
    
    if not ingredients:
        print("❌ No ingredients detected! Please check the recipe format.")
        return
    
    print("\n📋 Detected Ingredients:")
    for ing in ingredients:
        print(f"- {ing['quantity']} {ing['unit']} {ing['ingredient']}")
    
    print("\n⚡ Converting with Gemini...")
    result = convert_with_gemini(ingredients)
    
    if result:
        print("\n✅ Precision Conversions:")
        for item in result:
            print(f"{item['ingredient']}: {item['grams']}g ({item.get('notes', '')})")

# Test with your recipe
if __name__ == "__main__":
    recipe = """
    Classic Cookies:
    - 2 1/4 cups all-purpose flour
    - 1 teaspoon baking soda
    - 1 cup unsalted butter (softened)
    - 3/4 cup packed brown sugar
    - 2 cups chocolate chips
    """
    process_recipe(recipe)


Device set to use cpu


🔍 Extracting ingredients from recipe lines...

📋 Detected Ingredients:
- 2.25 cup all-purpose flour
- 1.0 teaspoon baking soda
- 1.0 cup unsalted butter (softened)
- 0.75 cup packed brown sugar
- 2.0 cup chocolate chips

⚡ Converting with Gemini...

✅ Precision Conversions:
all-purpose flour (loose): 281.25g ()
baking soda: 4.5g ()
unsalted butter (softened): 226g ()
packed brown sugar (packed): 165g ()
chocolate chips: 340g ()
